In [41]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten
import cv2
import os
from tqdm import tqdm
import numpy as np
# import pandas as pd
import re

In [4]:
images_paths = list(os.listdir('./Card Images/Preprocessed Images'))

In [12]:
imgs = []
for i in tqdm(images_paths):
    imgs.append(cv2.imread(f'./Card Images/Preprocessed Images/{i}'))

100%|██████████| 2757/2757 [00:49<00:00, 55.92it/s]


In [15]:
x = np.array(imgs)

In [17]:
x.shape

(2757, 540, 960, 3)

In [33]:
ranks = ['A', '2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K']
suits = ['C', 'D', 'H', 'S']


cards = [f'{rank}{suit}' for suit in suits for rank in ranks]
one_hot_matrix = np.eye(len(cards), dtype=int)

In [35]:
one_hot_matrix.shape

(52, 52)

In [42]:
def card_to_one_hot(card):
    if card in cards:
        index = cards.index(card)
        return one_hot_matrix[index]
    else:
        raise ValueError(f"Card '{card}' is not valid. Use format like 'AC' for Ace of Clubs.")

In [49]:
# pd.DataFrame(one_hot_matrix, index=cards, columns=cards)

In [43]:
cards_y = []
for i in images_paths[100:]:
    if 'JOKER' in i:
        continue
    match = re.match(r'^(A|J|K|Q|\d+)([CSHD])', i)
    face = match.group(1)
    suit = match.group(2)

    cards_y.append(f'{face}{suit}')

In [46]:
y = np.array([card_to_one_hot(i) for i in cards_y])

In [48]:
y.shape

(2606, 52)

In [ ]:
# could try relu or swish

In [50]:
model = Sequential()
img_size = (960, 540, 3)
model.add(Conv2D(input_shape=img_size,filters=64,kernel_size=(3,3),padding='same', activation='swish'))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding='same', activation='swish'))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(input_shape=(224, 244),filters=64,kernel_size=(3,3),padding='same', activation='swish')) # may need adjustment
model.add(Conv2D(filters=64,kernel_size=(3,3),padding='same', activation='swish'))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding='same', activation='swish'))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding='same', activation='swish'))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding='same', activation='swish'))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding='same', activation='swish'))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding='same', activation='swish'))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding='same', activation='swish'))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding='same', activation='swish'))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding='same', activation='swish'))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding='same', activation='swish'))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding='same', activation='swish'))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding='same', activation='swish'))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Dense(units=256))
model.add(Dense(units=52))

In [51]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 960, 540, 64)      1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 960, 540, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 480, 270, 64)     0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 480, 270, 64)      36928     
                                                                 
 conv2d_3 (Conv2D)           (None, 480, 270, 64)      36928     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 240, 135, 64)     0         
 2D)                                                    